In [1]:
#read dfs# import packages
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os 
from sklearn.metrics import accuracy_score, roc_curve, confusion_matrix, classification_report, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import RandomizedSearchCV
import sys 
from sklearn.model_selection import learning_curve
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler

In [2]:
train_X=pd.read_csv("../input/vol3-kopya/reduced_train_X.csv")
train_y=pd.read_csv("../input/vol3-kopya/train_y.csv")
normalized_test_df=pd.read_csv("../input/vol3-kopya/reduced_test_X.csv")
df_test1=pd.read_csv("../input/vol3-kopya/df_test1.csv")

In [3]:
train_X=train_X.drop(columns=train_X.columns[0])
train_y=train_y.drop(columns=train_y.columns[0])
normalized_test_df=normalized_test_df.drop(columns=normalized_test_df.columns[0])
df_test1=df_test1.drop(columns=df_test1.columns[0])

In [4]:
# cross-validation with 5 splits
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)

#Importing XGBM Classifier 
import lightgbm as lgb

#creating an extreme Gradient boosting instance
lgb_estimator = lgb.LGBMRegressor(#categorical_feature =[1,2,3,4,5 etc]
                                    early_stopping_round=100, 
                                    random_state=0, 
                                    device="GPU", 
                                    n_jobs =-1, 
                                    silent =0)

# parameters 
parameters = {
                "n_estimators": [100, 300, 500],
                "max_depth": [4, 5, 7],
                "subsample": [0.4, 0.7, 1],
                "reg_lambda":[1, 10, 100],
                "colsample_bytree": [0.4, 0.6, 1],
                "max_bin":[32, 64,128],
                'max_leaves': [15, 31, 63],
                "min_data_in_leaf":[1,10,100,1000],
}

In [5]:
# grid search for parameters
grid = RandomizedSearchCV(estimator=lgb_estimator, param_distributions=parameters, cv=cv, n_jobs=-1, n_iter=50)
grid.fit(train_X, train_y, eval_set = (train_X, train_y))

# print best scores
print("The best parameters are %s with a score of %0.4f"
      % (grid.best_params_, grid.best_score_))

# prediction results
y_pred = grid.predict(normalized_test_df)

/opt/conda/lib/python3.7/site-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	training's l2: 4.87613e+06
Training until validation scores don't improve for 100 rounds
[2]	training's l2: 4.34737e+06
[3]	training's l2: 4.13036e+06
[4]	training's l2: 3.76174e+06
[5]	training's l2: 3.61771e+06
[6]	training's l2: 3.50201e+06
[7]	training's l2: 3.41679e+06
[8]	training's l2: 3.31624e+06
[9]	training's l2: 3.1682e+06
[10]	training's l2: 3.15772e+06
[11]	training's l2: 2.89331e+06
[12]	training's l2: 2.66919e+06
[13]	training's l2: 2.53645e+06
[14]	training's l2: 2.44525e+06
[15]	training's l2: 2.38964e+06
[16]	training's l2: 2.30812e+06
[17]	training's l2: 2.26918e+06
[18]	training's l2: 2.22742e+06
[19]	training's l2: 2.1274e+06
[20]	training's l2: 2.07114e+06
[21]	training's l2: 2.00548e+06
[22]	training's l2: 1.97297e+06
[23]	training's l2: 1.90924e+06
[24]	training's l2: 1.88422e+06
[25]	training's l2: 1.83343e+06
[26]	training's l2: 1.81678e+06
[27]	training's l2: 1.77037e+06
[28]	training's l2: 1.74392e+06
[29]	training's l2: 1.68909e+06
[30]	training's l2: 1

In [6]:
# prediction results
pred_df=pd.DataFrame(zip(df_test1['sector_id'], y_pred), columns=['sector_id', 'predictions'])

In [7]:
submission_dff=pred_df.groupby('sector_id').sum()
ss=submission_dff.reset_index()

In [8]:
ss=ss.rename(columns={"sector_id": "Sektor_Id", "predictions": "Expected"})

In [9]:
ss.to_csv('out_random_search_lgbm.csv',index=False)
pred_df.to_csv('pred_df_random_search_lgbm', index=False)